In [ ]:
# Import necessary libraries for web scraping using Selenium and Scrapy
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from time import sleep
from scrapy import Selector
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# Set path to GeckoDriver (Firefox driver)
geckodriver = 'E:/Driver/geckodriver.exe'

# Define a custom user agent (optional)
user_agnt = 'Mozilla/5.0 (Windows NT 6.2; rv:81.0.1) Gecko/20100101 Firefox/81.0.1'

# Function to initialize Selenium Firefox driver
def init_driver(gecko_driver, load_images=True, user_agent=None, is_headless=False):
    firefox_profile = webdriver.FirefoxProfile()
    
    # Disable Flash plugin and sounds
    firefox_profile.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', False)
    firefox_profile.set_preference('media.volume_scale', '0.0')
    
    # Disable web notifications
    firefox_profile.set_preference('dom.webnotifications.enabled', False)
    
    # Optionally disable image loading to improve speed
    if not load_images:
        firefox_profile.set_preference('permissions.default.image', 2)
    
    # Apply custom user agent if provided
    if user_agent:
        firefox_profile.set_preference('general.useragent.override', user_agent)
    
    # Set Firefox options
    options = Options()
    options.headless = is_headless
    options.profile = firefox_profile
    
    # Start Firefox driver with the specified settings
    service = Service(executable_path=gecko_driver)
    driver = webdriver.Firefox(service=service, options=options)
    
    return driver

# Function to open a URL and return a Scrapy selector from the page source
def get_url(page_url, driver):
    driver.get(page_url)
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "listing-slider-container"))
        )
    except:
        print(f"Timeout waiting for page: {page_url}")
    
    return Selector(text=driver.page_source)

# Initialize the driver with custom settings
driver = init_driver(geckodriver, user_agent=user_agnt, load_images=False)


<>:7: SyntaxWarning: invalid escape sequence '\D'
<>:23: SyntaxWarning: invalid escape sequence '\D'
<>:7: SyntaxWarning: invalid escape sequence '\D'
<>:23: SyntaxWarning: invalid escape sequence '\D'
C:\Users\hp\AppData\Local\Temp\ipykernel_17272\259612683.py:7: SyntaxWarning: invalid escape sequence '\D'
  geckodriver='E:\Driver\geckodriver.exe'
C:\Users\hp\AppData\Local\Temp\ipykernel_17272\259612683.py:23: SyntaxWarning: invalid escape sequence '\D'
  service=Service(executable_path=f"E:\Driver\geckodriver.exe")


In [ ]:
li = []

# Loop over pages from 420 to 999
for i in range(420, 1000):
    base = 'https://aqarmap.com.eg'
    url = f'https://aqarmap.com.eg/ar/for-sale/apartment/?page={i}&minPrice=3750000&maxPrice=6000000'
    
    # Get the main listings on the current page
    main_page = get_url(url, driver)
    x = main_page.xpath('//a[contains(@class,"p-2x flex flex-col gap-y-2x")]')

    for item in x:
        dic = {}
        
        # Extract basic info from the listing card
        price = item.xpath('.//div/div/span/text()').extract()
        name = item.xpath('.//div/h2/text()').extract()
        link = item.xpath('.//@href').extract()
        location = item.xpath('./div[2]/div/p/text()').extract()

        if price: price = price[0]
        if name: name = name[0]
        
        new_url = base + link[0]
        
        # Go to the detailed page of the ad
        part_page = get_url(new_url, driver)
        
        # Extract apartment specs (area, rooms, bathrooms, finish state)
        xx = part_page.xpath('//section[contains(@class,"flex justify-between container-fluid flex-col-reverse gap-y-4x md:flex-row")]/div/div/div/p/text()').extract()
        
        if xx:
            area = xx[0] if len(xx) >= 1 else ''
            rooms = xx[2] if len(xx) >= 3 else ''
            pathroom = xx[4] if len(xx) >= 5 else ''
            state = xx[6] if len(xx) >= 7 else ''
        else:
            # Alternative XPath if structure is different
            xx = part_page.xpath('//div/ul')
            area = xx.xpath('.//li[.//text()[contains(., "المساحات")]]//text()').extract()
            rooms = xx.xpath('.//li[.//text()[contains(., "الغرف")]]//text()').extract()
            pathroom = xx.xpath('.//li[.//text()[contains(., "حمام")]]//text()').extract()
            state = xx.xpath('.//li[.//text()[contains(., "التشطيب")]]//text()').extract()
        
        # Extract property details: floor number, price per meter, year, view, payment method, ad type, ad ID
        zz = part_page.xpath('//section/div[contains(@class,"grid grid-cols-1 md:grid-cols-2 flex-row w-full rounded-[12px] overflow-hidden")]')
        if zz:
            roof = zz.xpath('./div[.//text()[contains(., "الدور")]]//text()').extract()
            roof = roof[1] if len(roof) >= 2 else ''
            price_per = zz.xpath('./div[.//text()[contains(., "سعر")]]//text()').extract()
            price_per = price_per[1] if len(price_per) >= 2 else ''
            date = zz.xpath('./div[.//text()[contains(., "سنة")]]//text()').extract()
            date = date[1] if len(date) >= 2 else ''
            viewe = zz.xpath('./div[.//text()[contains(., "تطل")]]//text()').extract()
            payment = zz.xpath('./div[.//text()[contains(., "دفع")]]//text()').extract()
            payment = payment[1] if len(payment) >= 2 else ''
            type_ads = zz.xpath('./div[.//text()[contains(., "نوع")]]//text()').extract()
            type_ads = type_ads[1] if len(type_ads) >= 2 else ''
            id_ad = zz.xpath('./div[.//text()[contains(., "رقم")]]//text()').extract()
            id_ad = id_ad[1] if len(id_ad) >= 2 else ''
        else:
            viewe = xx.xpath('.//li[.//text()[contains(., "تطل")]]//text()').extract()
            roof = xx.xpath('.//li[.//text()[contains(., "الدور")]]//text()').extract()
            price_per = xx.xpath('.//li[.//text()[contains(., "سعر")]]//text()').extract()
            payment = xx.xpath('.//li[.//text()[contains(., "طريقة")]]//text()').extract()
            type_ads = xx.xpath('.//li[.//text()[contains(., "المعلن")]]//text()').extract()
            id_ad = xx.xpath('.//li[.//text()[contains(., "رقم")]]//text()').extract()
            date = xx.xpath('.//li[.//text()[contains(., "سنة")]]//text()').extract()
        
        # Extract ad description
        describe = part_page.xpath('//section[contains(@class,"gap-y-3x container-fluid grid grid-cols-12")]//text()').extract()
        if not describe:
            disc = part_page.xpath('//div[contains(@id,"listingDescriptionText")]//text()')
            describe = " ".join(disc.extract()) if disc else ''
        
        # Extract rating scores if available
        bb = part_page.xpath('.//section[.//text()[contains(., "تقييم")]]')
        if bb:
            total = bb.xpath('./div/div[.//text()[contains(.,"عام")]]//text()').extract()
            level_area = bb.xpath('./div/div[.//text()[contains(.,"منطقة")]]//text()').extract()
            school = bb.xpath('./div/div[.//text()[contains(.,"مدارس")]]//text()').extract()
            services = bb.xpath('./div/div[.//text()[contains(.,"خدمات")]]//text()').extract()
            quitly = bb.xpath('./div/div[.//text()[contains(.,"هدوء")]]//text()').extract()
            loc = bb.xpath('./div/div[.//text()[contains(.,"موقع")]]//text()').extract()
            transports = bb.xpath('./div/div[.//text()[contains(.,"مواصلات")]]//text()').extract()
            market = bb.xpath('./div/div[.//text()[contains(.,"تسوق")]]//text()').extract()
            
            # Extract the actual values (2nd element usually)
            total = total[1] if len(total) >= 2 else ''
            level_area = level_area[1] if len(level_area) >= 2 else ''
            school = school[1] if len(school) >= 2 else ''
            services = services[1] if len(services) >= 2 else ''
            quitly = quitly[1] if len(quitly) >= 2 else ''
            loc = loc[1] if len(loc) >= 2 else ''
            transports = transports[1] if len(transports) >= 2 else ''
            market = market[1] if len(market) >= 2 else ''
        else:
            # Default empty values if ratings section not found
            total = level_area = school = services = quitly = loc = transports = market = ''

        # Collect all fields into one dictionary
        dic['name'] = name
        dic['price'] = price
        dic['location'] = location
        dic['link'] = new_url
        dic['area'] = area
        dic['rooms'] = rooms
        dic['pathroom'] = pathroom
        dic['state'] = state
        dic['roof'] = roof
        dic['price_per'] = price_per
        dic['date'] = date
        dic['viewe'] = viewe
        dic['payment'] = payment
        dic['type_ads'] = type_ads
        dic['id_ad'] = id_ad
        dic['total'] = total
        dic['level_area'] = level_area
        dic['school'] = school
        dic['services'] = services
        dic['quitly'] = quitly
        dic['loc'] = loc
        dic['transports'] = transports
        dic['market'] = market
        dic['describe'] = describe

        li.append(dic)

# Print the last page scraped
print(i)


NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:199:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:747:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:559:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.navigateTo@chrome://remote/content/marionette/driver.sys.mjs:1052:39
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20


In [16]:
print(i)

107


In [17]:
import pandas as pd
df=pd.DataFrame(li)
df

,name,price,location,link,area,rooms,pathroom,state,roof,price_per,...,id_ad,total,level_area,school,services,quitly,loc,transports,market,describe
0,شقه للبيع 130متر,"930,000\n جنيه",[القليوبية / شبرا الخيمة],https://aqarmap.com.eg/ar/listing/6030677-for-...,130 متر²,3 غرف,1 حمام,نصف تشطيب,2,"7,154 جنيه/متر²",...,EG-6030677,,,,,,,,,"[وصف الإعلان, قة للبيع برج أسانسير بشبرا الخيم..."
1,شقة للبيع اكسترا سوبر لوكس دور اخير بعزل حراري...,"720,000\n جنيه",[الاسكندرية / أبو يوسف],https://aqarmap.com.eg/ar/listing/6030379-for-...,120 متر²,3 غرف,1 حمام,اكسترا سوبر لوكس,7,"6,000 جنيه/متر²",...,EG-6030379,5,5,7,3,7,4,5,6,"[وصف الإعلان, ??فرصة للبيع شقة بالعجمي ابويوسف..."
2,شقه للبيع اكتوبر النخيل,"780,000\n جنيه",[الاسكندرية / شاطئ النخيل - ٦ اكتوبر],https://aqarmap.com.eg/ar/listing/6018232-for-...,100 متر²,2 غرف,1 حمام,سوبر لوكس,4,"7,800 جنيه/متر²",...,EG-6018232,,,,,,,,,"[وصف الإعلان, شقه للبيع الدور الرابع تصلح للسك..."
3,شقه للبيع ثاني شارع من البحر من الملك مباشر ال...,"820,000\n جنيه",[الاسكندرية / شاطئ النخيل - ٦ اكتوبر],https://aqarmap.com.eg/ar/listing/6024842-for-...,90 متر²,2 غرف,1 حمام,سوبر لوكس,الارضي,"9,111 جنيه/متر²",...,EG-6024842,,,,,,,,,"[وصف الإعلان, شقه للبيع ثاني شارع من البحر من ..."
4,شقة للبيع ٤ غرفه ١٨٠ متر عداد كهربه عداد مجمع ...,"320,000\n جنيه",[الاسكندرية / شارع ريتش هوم],https://aqarmap.com.eg/ar/listing/6024921-for-...,180 متر²,4 غرف,1 حمام,نصف تشطيب,7,"1,778 جنيه/متر²",...,EG-6024921,6,6,6,7,5,5,6,6,"[وصف الإعلان, شقة للبيع ٤ غرفه ١٨٠ متر عداد كه..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,شقه نص تشطيب خلف لاكازا ومدرسة سقارة اللبيني هرم,"900,000\n جنيه",[القاهرة الكبرى / اللبيني],https://aqarmap.com.eg/ar/listing/6014257-for-...,150 متر²,3 غرف,2 حمام,نصف تشطيب,11,"6,000 جنيه/متر²",...,EG-6014257,6.6,5.0,6.7,8.7,2.7,7.5,7.3,6.7,"[وصف الإعلان, شقة ب900 الف لفترة محدودة.فلو حا..."
739,شقة للبيع البيطاش اخر شارع الحي,"430,000\n جنيه",[الاسكندرية / شارع الحي],https://aqarmap.com.eg/ar/listing/6013907-for-...,120 متر²,3 غرف,1 حمام,لوكس,2,"3,583 جنيه/متر²",...,EG-6013907,6,6,6,7,5,5,6,6,"[وصف الإعلان, بسم الله توكلنا على الله شقة للب..."
740,شقه للبيع في ابو يوسف بحري دور رابع 110 متر,"770,000\n جنيه",[الاسكندرية / أبو يوسف],https://aqarmap.com.eg/ar/listing/6012819-for-...,110 متر²,3 غرف,1 حمام,سوبر لوكس,4,"7,000 جنيه/متر²",...,EG-6012819,5,5,7,3,7,4,5,6,"[وصف الإعلان, توكلنا علي الاه الله ?شقة للبيع ..."
741,شقه للبيع بسعر لقطه 90م,"650,000\n جنيه",[القاهرة الكبرى / حدائق حلوان],https://aqarmap.com.eg/ar/listing/6013237-for-...,90 متر²,2 غرف,1 حمام,لوكس,4,"7,222 جنيه/متر²",...,EG-6013237,6.4,4.0,7.3,7.7,5.0,5.1,7.0,6.0,"[وصف الإعلان, فرصة نادرة شقة للبيع في حدائق حل..."


In [ ]:
# Rename the columns of the DataFrame to more meaningful or consistent English names
x = [
    'ad_title', 'price', 'location', 'link', 'area', 'rooms', 'bathrooms', 'finish_status', 'floor_number',
    'price_per_meter', 'date', 'views', 'payment_method', 'ad_type', 'ad_id',
    'total_reviews', 'region_rating', 'schools_rating', 'services', 'quietness',
    'location_rating', 'transportation_rating', 'markets_rating', 'description',
]

# Apply the new column names
df.columns = x

# Define the desired order of columns for better structure and readability
cols = [
    'ad_id', 'ad_type', 'ad_title', 'location', 'area', 'rooms', 'bathrooms', 'finish_status', 'floor_number',
    'date', 'views', 'payment_method',
    'total_reviews', 'region_rating', 'schools_rating', 'services', 'quietness',
    'location_rating', 'transportation_rating', 'markets_rating', 'description', 'link', 'price_per_meter', 'price'
]

# Reorder the DataFrame columns
df = df[cols]


In [ ]:
df0=pd.read_csv('lastvirsion0.csv')
last=pd.concat([df0,df],ignore_index=True)
last.to_csv('lastvirsion0.csv', encoding="utf-8-sig",index=False)
